In [12]:
import torch
from torchvision import datasets, transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
import json

In [13]:
# Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
class PenultimateOutputsDataset(Dataset):
    def __init__(self, folder_path, device='cpu'):
        with open(f'{folder_path}/penultimate_layer_outputs/data.json') as data:
            self.data_tensors = json.load(data)
        self.file_list = [f'{folder_path}/penultimate_layer_outputs/penultimate_layer_outputs_{index}.pt' for index in range(self.data_tensors['last_file_indice']+1)]
        self.labels = torch.load(f'{folder_path}/penultimate_layer_labels.pt', map_location=device)
        self.length = self.data_tensors['min_size_segmentation']*self.data_tensors['last_file_indice']+torch.load(f'{folder_path}/penultimate_layer_outputs/penultimate_layer_outputs_{self.data_tensors["last_file_indice"]}.pt', map_location=device).shape[0]
        self.device = device
        self.data_cache = None
        
    def __len__(self):
        return self.length

    def __getitem__(self, index):
        if index < 0 or index >= self.length :
           raise ValueError(f'Index need to be between 0 and {self.length-1}')
        index_file = index//self.data_tensors['min_size_segmentation']
        index_in_file = index%self.data_tensors['min_size_segmentation']
        if (self.data_cache is None) or self.data_cache[0] != index_file :
            file = self.file_list[index_file]
            tensors = torch.load(file, map_location=self.device)
            self.data_cache = (index_file, tensors)
        tensor = self.data_cache[1][index_in_file]
        label = self.labels[index]
        return tensor, label


In [15]:
# root = '..\data\ImageNet_2012'
# split = 'train'
# transform = transforms.Compose([
#     transforms.Resize(256), # Met la taille du plus petit côté de l'image à 256 (l'autre reste proportionnel par rapport au rapport initial).
#     transforms.CenterCrop(224), # Sélectionne le carré de côté 224 à partir du centre de l'image
#     transforms.ToTensor(), # Convertir image PIL en tenseur avec des valeurs comprises dans [0,1]
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # normalise les valeurs du tenseur
# ])

In [16]:
outputs_folder = "../data/saved_outputs"
batch_size = 128
nb_epoque = 50
nb_classe = 1000
moyenne_erreur_classe_model = []
list_epoch = []
loss_epoch = []
dataset = PenultimateOutputsDataset(outputs_folder)
train_loader = DataLoader(dataset, batch_size=batch_size, pin_memory=True)

In [22]:
for i in range(nb_epoque):
    model = torch.load(f'../data/saved_models/epoch-{i}.pt', map_location='cpu')
    list_epoch.append(i)
    i_image = 0
    erreur_total_classe = [0 for j in range(nb_classe)]
    nb_image_classe = [0 for k in range(nb_classe)]
    total_loss = 0.0
    num_batches = 0
    
    model = model.to(DEVICE)
    model.eval()
    with torch.no_grad():
        for inputs, labels in tqdm(train_loader, desc="View"): 
            # Move inputs and labels to the device 
            inputs = inputs.to(DEVICE) 
            labels = labels.to(DEVICE) 
      
            # Forward pass 
            outputs = model(inputs) 
            erreur = torch.nn.functional.cross_entropy(inputs,labels,reduction='none')
            total_loss += torch.mean(erreur).item() #recupère la loss du batch
            num_batches += 1
            
            for val in erreur:
                erreur_total_classe[labels[i_image]] += val
                nb_image_classe[labels[i_image]] += 1
                i_image+=1
        loss_epoch.append(total_loss/num_batches)
        moyenne_erreur_classe_model.append(erreur_total_classe/nb_image_classe)
                       

NameError: name 'nb_epoque' is not defined

In [ ]:
moyenne_erreur_classe_model = torch.transpose(torch.tensor(moyenne_erreur_classe_model),0,1)
loss_epoch = torch.tensor(loss_epoch,0,1)
t_k = [torch.argmin(moyenne_erreur_classe_model[l]).item() for l in range(nb_classe)]
r_k = [torch.min(moyenne_erreur_classe_model[m]).item() for m in range(nb_classe)]
t = torch.argmin(loss_epoch)
r_t = torch.min(loss_epoch)
decalage = torch.abs(t_k - t)
regret = torch.abs(r_k - r_k[t])

In [ ]:
plt.plot(list_epoch,loss_epoch)
for n in range(nb_classe):
    plt.plot(list_epoch,moyenne_erreur_classe_model[n])